In [1]:
import pandas as pd
import numpy as np 

path = '/Users/miao.shang/Downloads/some score examples.csv'
df = pd.read_csv(path)

In [2]:
SOME_REF = {
    'pf': 1.2544,             # posting_frequency_c reference
    'er': 9.0481,             # engagement_rate reference
    'fc': (961, 467930),      # follower_count_c low/high refs
}
SOME_W = {
    'pf': 1.76,
    'er': 4.48,
    'fc': 3.76,
}

In [3]:
def compute_fc_score(value, low_ref, high_ref, weight):
    if pd.isna(value):
        return 0.0
    if value <= low_ref:
        return 0.0
    if value >= high_ref:
        return float(weight)
    return float((value - low_ref) / (high_ref - low_ref) * weight)

def compute_score(value, reference, weight):
    if pd.isna(value):
        return 0.0
    return float(weight if value >= reference else (value / reference) * weight)

def ensure_cols(df, cols, fill=0.0):
    """Make sure required columns exist; if missing, create them filled with 'fill'."""
    for c in cols:
        if c not in df.columns:
            df[c] = fill


In [4]:
import math
import pandas as pd

# --- Margin helpers ---
def m_ge(x, t, scale=1.0):
    return (x - t) / scale

def m_lt(x, t, scale=1.0):
    return (t - x) / scale

def m_range(x, lo, hi, scale=1.0):
    # distance to closest boundary (positive if inside)
    return min(x - lo, hi - x) / scale

def sigmoid(z):
    return 1.0 / (1.0 + math.exp(-z))

def rule_and(*margins):
    return min(margins)

def rule_or(*rule_margins):
    return max(rule_margins)

# --- Main: compute category + confidence ---
def some_category_and_confidence(row):
    pf_score = compute_score(row['Posting frequency'], SOME_REF['pf'], SOME_W['pf'])
    er_score = compute_score(row['Engagement Rate'], SOME_REF['er'], SOME_W['er'])
    fc_score = compute_fc_score(row['Follower count'], SOME_REF['fc'][0], SOME_REF['fc'][1], SOME_W['fc'])
    total_score = pf_score + er_score + fc_score

    # Suggested scales (tuneable but stable)
    S_PF = 0.5
    S_ER = 0.5
    S_FC = 0.5
    S_T  = 1.0

    # ----- Your rules as margins -----
    # Ignore rules:

    # R_ign_2: er < 0.7 and total < 1.22
    ign_2 = rule_and(
        m_lt(er_score, 0.7, S_ER),
        m_lt(total_score, 1.22, S_T),
    )
    # R_ign_3: 1.76 < total < 3.02  (strict on both sides)
    # We'll treat as inside (1.76, 3.02) using a range margin.
    ign_3 = m_range(total_score, 1.76, 3.02, S_T)

    ignore_margin = rule_or(ign_2, ign_3)

    # Launch rules:
    # R_lch_1: fc >= 0.5 and total >= 5
    lch_1 = rule_and(
        m_ge(fc_score, 0.5, S_FC),
        m_ge(total_score, 5.0, S_T),
    )
    # R_lch_2: 1.22 <= total < 5.2 and er >= 0.7 and fc >= 2.86
    lch_2 = rule_and(
        m_ge(total_score, 1.22, S_T),
        m_lt(total_score, 5.2, S_T),
        m_ge(er_score, 0.7, S_ER),
        m_ge(fc_score, 2.86, S_FC),
    )
    # R_lch_3: 3 <= total < 4.66 and 0.35 <= pf <= 1 and er >= 0.87
    lch_3 = rule_and(
        m_ge(total_score, 3.0, S_T),
        m_lt(total_score, 4.66, S_T),
        m_ge(pf_score, 0.35, S_PF),
        m_ge(1.0, pf_score, S_PF),     # pf <= 1  => margin = (1 - pf)
        m_ge(er_score, 0.87, S_ER),
    )

    launch_margin = rule_or(lch_1, lch_2, lch_3)

    # ----- Category assignment MUST match your original ordering -----
    if (fc_score >= 0.5 and total_score >= 5):
        category = 'launch'
        margin = lch_1
    elif (1.22 <= total_score < 5.2 and er_score >= 0.7 and fc_score >= 2.86):
        category = 'launch'
        margin = lch_2
    elif (er_score < 0.7 and total_score < 1.22):
        category = 'ignore'
        margin = ign_2
    elif (3 <= total_score < 4.66 and 0.35 <= pf_score <= 1 and er_score >= 0.87):
        category = 'launch'
        margin = lch_3
    elif (1.76 < total_score < 3.02):
        category = 'ignore'
        margin = ign_3
    else:
        category = 'review'
        # Review happens when you are NOT confidently in launch or ignore.
        # A good "review confidence" is: far from both launch and ignore regions.
        # If launch_margin/ignore_margin are negative, you're outside those regions.
        # Take the distance to the closest competing region:
        margin = min(-launch_margin, -ignore_margin)

    # Convert margin -> confidence in [0,1]
    # alpha controls steepness; 2~5 usually works well.
    alpha = 3.0
    conf = sigmoid(alpha * margin)

    return category, conf, pf_score, er_score, fc_score, total_score


In [8]:
tmp = df.apply(some_category_and_confidence, axis=1)

df['category_some_2_0'] = tmp.apply(lambda x: x[0])
df['confidence_some_2_0'] = tmp.apply(lambda x: x[1])
df['pf_score'] = tmp.apply(lambda x: x[2])
df['er_score'] = tmp.apply(lambda x: x[3])
df['fc_score'] = tmp.apply(lambda x: x[4])
df['total_score'] = tmp.apply(lambda x: x[5])


In [9]:
print(df)

                  Record Id Score updated date Overall recommendation  \
0   zcrm_699287000002448175         2025-12-11                 ignore   
1   zcrm_699287000002469899         2025-12-11                 review   
2   zcrm_699287000002502050         2025-12-11                 ignore   
3   zcrm_699287000002517349         2025-12-11                 review   
4   zcrm_699287000002539665         2025-12-11                 review   
5   zcrm_699287000002547466         2025-12-11                 review   
6   zcrm_699287000002564699         2025-12-11                 review   
7   zcrm_699287000002568711         2025-12-11                 launch   
8   zcrm_699287000002570300         2025-12-11                 ignore   
9   zcrm_699287000002578184         2025-12-11                 launch   
10  zcrm_699287000002581997         2025-12-11                 review   
11  zcrm_699287000002582206         2025-12-11                 review   
12  zcrm_699287000009682512         2025-12-11     

In [19]:
BASE = {"ignore": 0, "review":2, "launch": 6}
df["priority_score"] = df["category_some_2_0"].map(BASE) + 4 * df["confidence_some_2_0"]


In [20]:
print(df[["Overall recommendation","SoMe score","category_some_2_0","priority_score"]])

   Overall recommendation  SoMe score category_some_2_0  priority_score
0                  ignore        2.57            review        5.057725
1                  review        4.26            review        5.493720
2                  ignore        1.77            ignore        3.471241
3                  review        6.88            review        4.452098
4                  review        6.16            review        4.569118
5                  review        7.57            review        5.686533
6                  review        6.97            review        4.246758
7                  launch        6.36            launch        8.959961
8                  ignore        3.46            review        5.529791
9                  launch        8.16            launch        9.184792
10                 review        5.60            ignore        2.708555
11                 review        6.30            ignore        3.080035
12                 launch        8.91            review        5

In [21]:
df.to_csv('some_score_evaluation_updated.csv', index=False)